In [ ]:
import os
HOME_FOLDER = "/Users/luna/workspace/TensorFlow-Tutorial/"

#create train folders
MNIST_FOLDER = os.path.join(HOME_FOLDER, "tf_mnist")
TRAIN_1_FOLDER = os.path.join(MNIST_FOLDER, "train-1")
TRAIN_2_FOLDER = os.path.join(MNIST_FOLDER, "train-2")
TRAIN_3_FOLDER = os.path.join(MNIST_FOLDER, "train-3")
if not os.path.isdir(MNIST_FOLDER):
    os.mkdir(MNIST_FOLDER)
if not os.path.isdir(TRAIN_1_FOLDER):
    os.mkdir(TRAIN_1_FOLDER)
if not os.path.isdir(TRAIN_2_FOLDER):
    os.mkdir(TRAIN_2_FOLDER)
if not os.path.isdir(TRAIN_3_FOLDER):
    os.mkdir(TRAIN_3_FOLDER)
    

import os
import tensorflow as tf
import sys
import urllib

if sys.version_info[0] >= 3:
    from urllib.request import urlretrieve
else:
    from urllib import urlretrieve

GITHUB_URL ='https://raw.githubusercontent.com/mamcgrath/TensorBoard-TF-Dev-Summit-Tutorial/master/'
    
    
### MNIST EMBEDDINGS ###
mnist = tf.contrib.learn.datasets.mnist.read_data_sets(train_dir=MNIST_FOLDER
                                                       + '/data', one_hot=True)
### Get a sprite and labels file for the embedding projector ###
urlretrieve(GITHUB_URL + 'labels_1024.tsv', MNIST_FOLDER + '/labels_1024.tsv')
urlretrieve(GITHUB_URL + 'sprite_1024.png', MNIST_FOLDER + '/sprite_1024.png')


#Setting up layers in CNN and training
def conv_layer(x, channels_in, channels_out, name="conv"):
    ###########################################
    #@TODO: Enter Code Here - use the name
    # paramatere from the function
    
    with tf.name_scope(name):
    ###########################################
    
        # add a name attribute to w
        w = tf.Variable(tf.zeros([5, 5, channels_in, 
                                 channels_out]), name="W")
    
        # add a name attribute to b
        b = tf.Variable(tf.zeros([channels_out]), name="B")
        
        conv = tf.nn.conv2d(x, w, 
                            strides=[1, 1, 1, 1], 
                            padding="SAME")
        
        act = tf.nn.relu(conv + b)
        
        ###########################################
        #@TODO: Enter Code Here - add histogram
        # summaries to w, b and act, with names
        # "weights", "biases" and "activations"
        # respectively
        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activations", act)
        ###########################################
        return tf.nn.max_pool(act, ksize=[1, 2, 2, 1], 
                              strides=[1, 2, 2, 1], padding="SAME")


# Add a name attribute to fc_layer. name it "fc"
def fc_layer(x, channels_in, channels_out, name="fc"):
    ###########################################
    #@TODO: Enter Code Here - use the name
    # paramatere from the function
    
    with tf.name_scope(name):
    ###########################################
    
        #@TODO: add a name attribute to w. name it "W"
        w = tf.Variable(tf.zeros([channels_in, 
                                  channels_out]), name="W")
        
        #@TODO: add a name attribute to b. name it "B"
        b = tf.Variable(tf.zeros([channels_out]), name="B")
        
        act = tf.nn.relu(tf.matmul(x, w) + b)
        
        ###########################################
        #@TODO: Enter Code Here - add histogram
        # summaries to w, b and act, with names
        # "weights", "biases" and "activations"
        # respectively
        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activations", act)
        ###########################################
        return act

# Add a name attribute to fc_layer. name it "fc_no_relu"
def fc_no_relu(x, channels_in, channels_out, name="fc_no_relu"):
    ###########################################
    #@TODO: Enter Code Here - use the name
    # paramatere from the function
    
    with tf.name_scope(name):
    ###########################################
    
        #@TODO: add a name attribute to w. name it "W"
        w = tf.Variable(tf.zeros([channels_in, 
                                  channels_out]), name="W")
        
        #@TODO: add a name attribute to b. name it "B"
        b = tf.Variable(tf.zeros([channels_out]), name="B")
        
        
        out = tf.matmul(x, w) + b
        
        ###########################################
        #@TODO: Enter Code Here - add histogram
        # summaries to w, b and act, with names
        # "weights", "biases" and "activations"
        # respectively
        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)
        ###########################################
        return out

def mnist_model(learning_rate, use_two_conv, use_two_fc,
                num_iterations, batch_size, hparam, log_dir):
    
    tf.reset_default_graph()
    sess = tf.Session()
    
    # Setup placeholders, and reshape the data
    #@TODO: Add name attribute to x. name it "x"
    x = tf.placeholder(tf.float32, shape=[None, 784], name="x")
    
    #@TODO: Add name attribute to y. name it "labels"
    y = tf.placeholder(tf.float32, shape=[None, 10], name="labels")
    
    x_image = tf.reshape(x, [-1, 28, 28, 1])
    
    ###########################################
    #@TODO: Enter Code Here - add image summary
    # of x_image using tf.summary.image.
    # name it "input"
    tf.summary.image('input', x_image, 3)
    ###########################################
    
    if use_two_conv:
        #@TODO: pass the name "conv1"
        conv1 = conv_layer(x_image, 1, 32, "conv1")
        
        #@TODO: pass the name "conv2"
        conv_out = conv_layer(conv1, 32, 64, "conv2")
    else:
        
        #@TODO: pass the name "conv"
        conv1 = conv_layer(x_image, 1, 64, "conv")
        conv_out = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], 
                                  strides=[1, 2, 2, 1], 
                                  padding="SAME")

    flattened = tf.reshape(conv_out, [-1, 7 * 7 * 64])
    keep_prob = tf.placeholder(tf.float32)
    
    if use_two_fc:
        
        #@TODO: pass the name "fc1"
        fc1 = fc_layer(flattened, 7 * 7 * 64, 1024, "fc1")
        
        embedding_input = fc1
        embedding_size = 1024
        
        # add dropout
        fc1 = tf.nn.dropout(fc1, keep_prob)
        
        #@TODO: pass the name "fc2"
        logits = fc_no_relu(fc1, 1024, 10, "fc2")
        
    else:
        embedding_input = flattened
        embedding_size = 7*7*64
        
        #@TODO: pass the name "fc"
        logits = fc_no_relu(flattened, 7*7*64, 10, "fc")
        logits = tf.nn.dropout(logits, keep_prob)
    
    ###########################################
    #@TODO: Enter Code Here - add a name scope
    #to xent. name it "xent"
    
    with tf.name_scope("xent"):
    ###########################################
    
        xent = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(
                logits=logits, labels=y), name="xent")
        
        ###########################################
        #@TODO: Enter Code Here - add scalar
        # summaries to xent, with name "xent"
        tf.summary.scalar("loss", xent)
        ###########################################
        
    ###########################################
    #@TODO: Enter Code Here - add a name scope
    #to train_step. name it "train"
    with tf.name_scope("train"):
    ###########################################
    
        train_step = tf.train.AdamOptimizer(learning_rate).minimize(xent)

    ###########################################
    #@TODO: Enter Code Here - add a name scope
    #to prediction and accuracy. name it 
    #"accuracy"
    with tf.name_scope("accuracy"):
    ###########################################
    
        correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
        ###########################################
        #@TODO: Enter Code Here - add scalar
        # summaries to accuracy, with name "accuracy"
        tf.summary.scalar("accuracy", accuracy)
        ###########################################
        
    # merge all summaries    
    summ = tf.summary.merge_all()
    
    # Embedding visualization
    embedding = tf.Variable(tf.zeros([1024, embedding_size]), 
                            name="test_embedding")
    assignment = embedding.assign(embedding_input)
    saver = tf.train.Saver()

    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter(log_dir + "/" + hparam)
    writer.add_graph(sess.graph)
    
    config = tf.contrib.tensorboard.plugins.projector.ProjectorConfig()
    embedding_config = config.embeddings.add()
    embedding_config.tensor_name = embedding.name
    embedding_config.sprite.image_path = MNIST_FOLDER + '/sprite_1024.png'
    embedding_config.metadata_path = MNIST_FOLDER + '/labels_1024.tsv'
      
    # Specify the width and height of a single thumbnail.
    embedding_config.sprite.single_image_dim.extend([28, 28])
    tf.contrib.tensorboard.plugins.projector.visualize_embeddings(writer, config)

    for i in range(num_iterations+1):
        batch = mnist.train.next_batch(batch_size)
        if i % 10 == 0:
            [loss, train_accuracy, s] = sess.run([xent, accuracy, summ], 
                                           feed_dict={x: batch[0], 
                                                      y: batch[1],
                                                     keep_prob: 1.0})
            print("step %d, minibatch loss %g, training accuracy %g" 
                  % (i, loss, train_accuracy))
            writer.add_summary(s, i)
        train_step.run(session=sess, feed_dict={x: batch[0], 
                                                y: batch[1], 
                                                keep_prob: 0.5})
        if i % 100 == 0:
            sess.run(assignment, feed_dict={x: mnist.test.images[:1024], 
                                            y: mnist.test.labels[:1024],
                                           keep_prob: 0.5})
            saver.save(sess, os.path.join(log_dir, "model.ckpt"), i)
            print('test accuracy %g' % accuracy.eval(session=sess, feed_dict={
                x: mnist.test.images, 
                y: mnist.test.labels, 
                keep_prob: 1.0}))

def make_hparam_string(learning_rate, 
                       use_two_fc, 
                       use_two_conv,
                       num_iterations,
                       batch_size):
    conv_param = "conv=2" if use_two_conv else "conv=1"
    fc_param = "fc=2" if use_two_fc else "fc=1"
    num_iteration_param = "num_iter=" + str(num_iterations)
    batch_size_param = "batch_size=" + str(batch_size)
    return "lr_%.0E,%s,%s,%s,%s" % (learning_rate, conv_param, 
                                    fc_param, num_iteration_param,
                                   batch_size_param)


for learning_rate in [1E-4, 1E-3]:
    # Include "False" as a value to try different model architectures
    for use_two_fc in [True, False]:
        # Include "False" as a value to try different model architectures
          for use_two_conv in [True, False]:
            # Include different batch size
            for batch_size in [100]:
                # Include different number of iterations
                for num_iterations in [200]:
                    
                    hparam = make_hparam_string(learning_rate, use_two_fc, 
                                                use_two_conv, 
                                                num_iterations, 
                                                batch_size)
                    print('Starting run for %s' % hparam)

                    # Actually run with the new settings
                    mnist_model(learning_rate, use_two_fc, use_two_conv, 
                                num_iterations, batch_size, hparam, TRAIN_3_FOLDER)
                    
                    
#Setting up tensorboard and running on anaconda
"""
tensorboard --logdir=/Users/luna/workspace/TensorFlow-Tutorial/tf_mnist/train-2
Click on Graphs tab:
On the left hand side, you will see a field called "Run"
Select the "lr_1E-04,conv=2,fc=2,2000,batch_size=100" run

About each Visualization on Tensorboard:
Weights Histogram: The historgram looks like a normal distribution. With increasing time steps, the distribution seems to approach a bell curve. This might signify that our model is generalizing well.

Biases Histogram: It is seen that at early stages of the training, the biases are big and it decreases gradually with increasing time steps.

Activations Histogram: It is seen that the activation histogram has a spike at bin value of 0.0934 which signifies the threshold of our neuron.

"""